## Table of Contents:
1. [Load in the Data](#first)
2. [Create Text Matrices](#second)
3. [Vectorize Text Matrices](#third)
4. [Examing Distribution of All Variables](#fourth)
5. [Exploring Demographic Features](#fifth)
6. [Exploring Work Logistics Features](#sixth)
7. [Apriori Algorithm](#seventh)


In [ ]:
%load_ext autoreload
%autoreload 2



In [ ]:
#Assuming you are using Google Colab, running this cell will clone the entire repository into Colab
!git clone https://github.com/RiceD2KLab/DSCI400_Sp23.git

In [ ]:
#Go to repository folder containing Jupyter notebooks and python files to run
%cd DSCI400_Sp23

In [ ]:
#Import all functions defined for data cleaning and data vectorization
from data_cleaning_functions import *
from data_vectorization_function import *

In [ ]:
#Import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Load in the Data <a name="first"></a>

Before loading in our data, we wanted to find the shared questions (features) across all years. Using the 2016 csv OSMI dataset, we assigned an integer to each question. For each question from 2016, we went through the 2017-2021 datasets to find questions that asked the same things (even if the wording was slightly different) and assigned shared questions across all years the same integer. If a question did not appear in all years from 2016-2021 surveys, that question did not get assigned an integer. A row containing these integers was added as the first row in each year's dataset.


In [ ]:
#Load in raw data csv's updated with row containing number labels
#Download 2016-2021 csv files from data folder in github repository
#Click on "Files" tab on left-hand side and upload 2016-2021 csv files 

df2016Updated = pd.read_csv('/content/2016updated.csv')
df2017Updated = pd.read_csv('/content/2017updated.csv')
df2018Updated = pd.read_csv('/content/2018updated.csv')
df2019Updated = pd.read_csv('/content/2019updated.csv')
df2020Updated = pd.read_csv('/content/2020updated.csv')
df2021Updated = pd.read_csv('/content/2021updated.csv')

#Create Text Matrices <a name="second"></a>

Use makeTextMatrix, makePreCovidMatrix, and makeDuringCovidMatrix functions defined in data_cleaning_functions python file to create combined text, precovid, and during covid dataframes.



In [ ]:
#Create combined text matrix of all 6 years
dfCombined = makeTextMatrix(df2016Updated, df2017Updated, df2018Updated, df2019Updated, df2020Updated, df2021Updated)

#Create precovid matrix concatenating 2016-2019 years' data
dfPreCovid = makePreCovidMatrix(df2016Updated, df2017Updated, df2018Updated, df2019Updated)

#Create during matrix concatenating 2020-2021 years' data
dfDuringCovid = makeDuringCovidMatrix(df2020Updated, df2021Updated)

#filter out features with more than 30% null values
dfCombinedFinal, dfPreCovidFinal, dfDuringCovidFinal = filterNull(dfCombined, dfPreCovid, dfDuringCovid)


#Vectorize Text Matrices <a name="third"></a>

For each of the three dataframes, we convert the categorical text responses into into numerical ones.



In [ ]:
dfCombinedVectorized = vectorizeDataframe(dfCombinedFinal)
dfCombinedVectorized

In [ ]:
dfPreCovidVectorized = vectorizeDataframe(dfPreCovidFinal)
dfPreCovidVectorized

In [ ]:
dfDuringCovidVectorized = vectorizeDataframe(dfDuringCovidFinal)
dfDuringCovidVectorized

The dataframe representing years during covid (2020-2021) is extremely small after removing rows with NA values. It only contains 161 responses, which is significantly less than the 1879 responses in the precovid dataframe. Thus, moving forward, we will focus our efforts on data exploration and machine learning modeling on the combined dataframe with all 6 years. To accurately explore the effect of Covid-19 on mental health in tech workers by comparing pre-covid vs during covid data, in the future, we would need to add a lot more responses to the during covid dataframe, perhaps collected from a source outside of OSMI.

#Examing Distribution of All Variables <a name="fourth"></a>

For each variable in the combined dataframe, we will look at a histogram of its distribution (i.e., the distribution of responses to each survey question). 



In [ ]:
#Graph distribution of responses in output variable
column_name = '45'

# Count the occurrences of 0s and 1s in the selected column
value_counts = dfCombinedVectorized[column_name].value_counts()

colors = ['red', 'green']


plt.bar(value_counts.index, value_counts.values, color=colors)
plt.xlabel('Response')
plt.ylabel('Frequency')
plt.title(f'Histogram of "Do you currently have a mental health disorder?"')
plt.xticks(list(value_counts.index), ['No', 'Yes'])  # Set the x-axis ticks to display as '0' and '1'
plt.show()

We first focused on the distribution of the output variable, "Do you currently have a mental health disorder?" We can see that the output variable was successfully made binary. Because "No" represents no, don't know, maybe, and possibly, it makes sense why there are no "No" responses than "Yes" responses. However, the distribution is not terribly skewed or uneven. 

In [ ]:
#Create a histogram of each feature
for col in dfCombinedVectorized.columns:
    plt.figure(figsize=(4, 3))  # create a larger figure
    plt.hist(dfCombinedVectorized[col], bins=10, color='skyblue', alpha=0.75)  # customize color and transparency
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(axis='y', alpha=0.75)  # add a grid
    plt.xticks(rotation=45)
    plt.show()

Looking at histograms of 1 and 25, we noticed that there was only 1 response type for both. Thus, we dropped these features. Feature 1 asks "Are you self-employed?" and feature 25 asks "Do you have previous employers?". Answering yes to being self-empoyed and/or no to having previous employers prevents you from being able to answer many of the other survey questions. It then makes sense that after dropping rows with NA values, features 1 and 25 only have 1 response type.

In [ ]:
#dropping features 1 and 25 from the combined dataframe
dfCombinedVectorized.drop('1', axis=1, inplace=True)
dfCombinedVectorized.drop('25', axis=1, inplace=True)
dfCombinedVectorized

Looking more into the exact wording of the survey questions left, we noticed that features 51 and 52 can only be answered if a responsdent has a mental health issue. Feature 51 asks "If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?" and feature 52 asks "If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?". Thus, they are not helpful in building models for predicting whether a tech worker currently has a mental health disorder. 

In [ ]:
#dropping features 51 and 52 from the combined dataframe
dfCombinedVectorized.drop('51', axis=1, inplace=True)
dfCombinedVectorized.drop('52', axis=1, inplace=True)
dfCombinedVectorized

#Exploring Demographic Features <a name="fifth"></a>

We wanted to explore demographic features like age and gender more, as they can inform tech companies of certain demographic groups that are at higher risk to have mental health issues. 



In [ ]:
#Graphing percentage of respondents who have mental health issues when grouped by age range

outputColumn = '45' 
ageColumn = '53'

# Filter the rows where the output variable is 'yes'
yesResponses = dfCombinedVectorized[dfCombinedVectorized[outputColumn] == 1]

# Calculate the total number of responses
totalResponses = len(dfCombinedVectorized)

# Group the yes responses by age range
ageGroups = yesResponses.groupby(pd.cut(yesResponses[ageColumn], bins=[18, 25, 35, 45, 55, 65, 120], right=False)).size()

# Calculate the percentage of yes responses out of total responses for each age group
percbyAge = ageGroups / totalResponses

# Create a bar plot of the percentage by age range
plt.bar(percbyAge.index.astype(str), percbyAge.values)

# Set the labels and title
plt.xlabel('Age Range')
plt.ylabel('Proportion Having Mental Health Issue')
plt.title('Proportion of Respondents Having Mental Health Issue by Age')

# Show the bar plot
plt.show()

In [ ]:
 #Graphing percentage of respondents who have mental health issues when grouped by gender

# Define the column names
outputColumn = '45' 
genderColumn = '54'  # Replace with the name of your gender column

# Filter the rows where the output variable is 'yes'
yesResponses = dfCombinedVectorized[dfCombinedVectorized[outputColumn] == 1]

# Calculate the total number of responses
totalResponses = len(dfCombinedVectorized)

# Group the yes responses by gender
genderGroups = yesResponses[genderColumn].value_counts()

# Calculate the percentage of yes responses out of total responses for each gender
percbyGender = genderGroups / totalResponses

# Create a bar plot of the percentage by gender
plt.bar(percbyGender.index, percbyGender.values)

# Set the labels and title
plt.xlabel('Gender')
plt.ylabel('Proportion Having Mental Health Issue')
plt.title('Proportion of Respondents Having Mental Health Issue by Gender')

# Define custom x-axis tick labels
plt.xticks(ticks=percbyGender.index, labels=['Male', 'Female', 'Other'])  

# Show the bar plot
plt.show()


#Exploring Work Logistics Features <a name="sixth"></a>

We wanted to explore some work logistics features, as they can inform tech companies of how to best structure their workplace environment and culture to improve mental health for tech workers. 



In [ ]:
#Feature 8 is "Does your employer offer resources to learn more about mental health concerns and options for seeking help?"
#Graphing percentage of respondents who have mental health issues when grouped by response to feature 8


# Define the column names
outputColumn = '45'  
feature8Column = '8' 

# Filter the rows where the output variable is 'yes'
yesResponses = dfCombinedVectorized[dfCombinedVectorized[outputColumn] == 1]

# Calculate the total number of responses
totalResponses = len(dfCombinedVectorized)

# Group the yes responses by response to feature 8
feature8Groups = yesResponses[feature8Column].value_counts()

# Calculate the percentage of yes responses out of total responses for each response option to feature 8
percbyFeature8 = feature8Groups / totalResponses

# Create a bar plot of the percentage by country
plt.bar(percbyFeature8.index, percbyFeature8.values)

# Set the labels and title
plt.xlabel('Response')
plt.ylabel('Proportion Having Mental Health Issue')
plt.title('Mental Health Prevalence and Employer Provided Resources')

# Define custom x-axis tick labels
plt.xticks(ticks=percbyFeature8.index, labels=['I don\'t know', 'No', 'Yes'])

# Show the bar plot
plt.show()


In [ ]:
#Feature 10 is "If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:"
#Graphing percentage of respondents who have mental health issues when grouped by response to feature 10

# Define the column names
outputColumn = '45'  
feature10Column = '10' 

# Filter the rows where the output variable is 'yes'
yesResponses = dfCombinedVectorized[dfCombinedVectorized[outputColumn] == 1]

# Calculate the total number of responses
totalResponses = len(dfCombinedVectorized)

# Group the yes responses by response to feature 10
feature10Groups = yesResponses[feature10Column].value_counts()

# Calculate the percentage of yes responses out of total responses for each response option to feature 10
percbyFeature10 = feature10Groups / totalResponses

# Create a bar plot of the percentage by country
plt.bar(percbyFeature10.index, percbyFeature10.values)

# Set the labels and title
plt.xlabel('Response')
plt.ylabel('Proportion Having Mental Health Issue')
plt.title('Mental Health Prevalence and Asking for Leave')

# Define custom x-axis tick labels
plt.xticks(ticks=percbyFeature10.index, labels=['I don\'t know', 'Very easy', 'Somewhat easy', 'Neither easy nor difficult', 'Somewhat difficult', 'Difficult', 'Very difficult'], rotation=90)

# Show the bar plot
plt.show()

In [ ]:
#Feature 28 is "Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?"
#Graphing percentage of respondents who have mental health issues when grouped by response to feature 28

# Define the column names
outputColumn = '45'  
feature28Column = '28' 

# Filter the rows where the output variable is 'yes'
yesResponses = dfCombinedVectorized[dfCombinedVectorized[outputColumn] == 1]

# Calculate the total number of responses
totalResponses = len(dfCombinedVectorized)

# Group the yes responses by response to feature 28
feature28Groups = yesResponses[feature28Column].value_counts()

# Calculate the percentage of yes responses out of total responses for each response option to feature 28
percbyFeature28 = feature28Groups / totalResponses

# Create a bar plot of the percentage by country
plt.bar(percbyFeature28.index, percbyFeature28.values)

# Set the labels and title
plt.xlabel('Response')
plt.ylabel('Proportion Having Mental Health Issue')
plt.title('Mental Health Prevalence and Mental Health Discussion')

# Define custom x-axis tick labels 
plt.xticks(ticks=percbyFeature28.index, labels=['None did', 'Some did', 'Yes, they all did', 'I don\'t know'], rotation=90)

# Show the bar plot
plt.show()

#Apriori Algorithm <a name="seventh"></a>

We leveraged the apriori algorithm, an association algorithm that identifies certain responses to specific survey questions that frequently co-occur together. We specifically wanted to find responses that co-occur with answering "yes" to currently having a mental health disorder.



In [ ]:
!pip install efficient-apriori
from efficient_apriori import apriori, Rule

In [ ]:
#To run the apriori algorithm, we need to convert our dataframe into a list of tupules, where each tupule represents a respondent's answers to each question
transactionMatrix = []
for i in range (0,2040):
  rowElement = dfCombinedVectorized.iloc[i]
  temporaryList= []
  for colIndex in range (0,26):
    eachElement = rowElement[colIndex]
    questionNumber = dfCombinedVectorized.columns[colIndex]
    temporaryList.append(str(questionNumber) + "= " + str(eachElement)  )
  transactionMatrix.append(tuple(temporaryList))

In [ ]:
#Running apriori algorithm with minimum support and confidence thresholds low enough to generate a substantial list of association rules
itemsets, rules = apriori(transactionMatrix, min_support=0.1, min_confidence=0.6)

#Filter for rules where answering yes to currently having a mental health disorder is on the right hand side of the association rule
rulesImportant = [rule for rule in rules if '45= 1' in rule.rhs]

#Sort important rules by descending order of support values
rulesSorted = sorted(rulesImportant, key=lambda x: x.support, reverse=True)
for rule in rulesSorted:
    print(rule)
    print(rule.support)
    print("=====================================")

The association rules with the highest support values, although having good association strengths, do not reveal anything too useful or interesting. They mainly show rules suggesting a strong association between having a medical history of mental health and currently having a mental health disorder.

In [ ]:
#Previous histogram showing distribution of feature 28 suggests that not having a mental health discussion with previous employers associates with greater percentage of respondents having mental health issues
#Filtering rulesImportant even further by looking for rules where answering yes to mental health being discussed is on the left side
rulesWorkLog = [rule for rule in rulesImportant if '28= 1.0' in rule.lhs]
rulesSorted = sorted(rulesWorkLog, key=lambda x: x.support, reverse=True)
for rule in rulesSorted:
    print(rule)
    print(rule.support)
    print("=====================================")

In [ ]:
#Filtering rulesImportant even further by looking for rules where answering no to mental health being discussed is on the left side
rulesWorkLog = [rule for rule in rulesImportant if '28= 0.0' in rule.lhs]
rulesSorted = sorted(rulesWorkLog, key=lambda x: x.support, reverse=True)
for rule in rulesSorted:
    print(rule)
    print(rule.support)
    print("=====================================")